In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains


from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException

In [ ]:
import numpy as np

In [ ]:
driver.quit()

In [ ]:
driver = webdriver.Chrome('/Users/jonghyunkim/repo/Tristan/chromedriver')
driver.maximize_window()

In [ ]:
url = 'https://sdgs.un.org/partnerships/browse'
driver.get(url)

In [ ]:
def scroll_down_and_click(driver, element):
    while True:
        try:
            element.click()
            break
        except:
            driver.find_element(By.CSS_SELECTOR, 'body').send_keys(Keys.ARROW_DOWN)

In [ ]:
def gpt_click(driver, element):
    # Find all elements with the specified class


    # Maximum wait time per element
    max_wait_time = 10


    end_time = time.time() + max_wait_time
    while time.time() < end_time:
        try:
            # Try to move to the element and click it
            WebDriverWait(driver, 2).until(EC.element_to_be_clickable(element))
            element.click()
            # print(f"Element {element} is clickable and clicked")
            break
        except Exception as e:
            # print(f"Element {element} not clickable yet: {e}")
            # Scroll down
            # driver.execute_script("window.scrollBy(0, 100);")
            driver.find_element(By.CSS_SELECTOR, 'body').send_keys(Keys.ARROW_DOWN)
            time.sleep(0.5)  # Adding a small delay to prevent overwhelming the browser

    # # If the element is not found or clickable within the maximum wait time, continue to the next element
    # if time.time() >= end_time:
    #     raise

In [ ]:
import pandas as pd

In [ ]:
reference_table = pd.read_excel('SDGS_on_process.xlsx', index_col=False)
reference_table.head()

In [ ]:
ignored_exceptions=(NoSuchElementException,StaleElementReferenceException,)

In [ ]:
len(reference_table)

In [ ]:
# o = 1

# while True:
#     element = driver.find_elements(By.CLASS_NAME, 'page-item')[-2]
#     gpt_click(driver, element)
#     o += 1

#     if o == 234:
#         break

In [ ]:
processing = driver.find_element(By.CLASS_NAME, 'offset-3').text.strip()
total_number =int(processing.split()[-1])

current_number = len(reference_table)

if total_number > current_number:
    diff = total_number - current_number
else:
    diff = 0

current_list = list(reference_table['Case_Num'])
new_list = []
for nums in current_list:
    new_list.append(nums+diff)

not_founds_index = list(reference_table[reference_table['Case_Name'] == 'Page Not found']['Case_Num'])

new_error_list = []
for index in  not_founds_index:
    new_error_list.append(index + diff)

[4203, 4396, 7692]

In [ ]:
total_number

In [ ]:
new_error_list

In [ ]:
case_numbers = []
cases_on_main_page = []
case_names = []
descrip_tab = []
sdgs_tab = []
deliver_tab = []
resource_mobilized_tab = []
progress_tab = []

right_hand_info = []

k = 1
numbers = 1

errors = []

while True:

    cases_in_page = driver.find_element(By.CLASS_NAME, 'view-content').find_elements(By.CLASS_NAME, 'views-row')

    for case in cases_in_page:
        # 가끔 클릭했는데 페이지가 안넘어가는 애들이 있어서 얘네는 따로 저장해주는 걸로.
        if numbers in new_error_list:
            errors.append((numbers, 'Page Not found'))
            numbers += 1
            continue

        if numbers in new_list:
            if reference_table['Description_Intro'][numbers-1-diff] != 'error':
                numbers += 1
                continue
        
        # if case.find_element(By.CLASS_NAME, 'views-field').text.strip() in list(ref['Case_Name']):
        #     numbers += 1
        #     continue
            
        try:
            case2 = case.find_elements(By.CLASS_NAME, 'views-field')[0].find_element(By.CLASS_NAME, 'field-content')
            cases_on_main_page.append(case2.text.strip())
            gpt_click(driver, case2)
            time.sleep(5)
            # WebDriverWait(driver, 3,ignored_exceptions=ignored_exceptions).until(EC.presence_of_element_located((By.CLASS_NAME, 'separator')))
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            # 케이스의 이름 - identifier
            case_name = soup.find(class_ = 'separator-bottom mt-5').text.strip()
        except:
            try:
                errors.append((numbers, case2.text.strip()))
            except StaleElementReferenceException:
                errors.append((numbers, 'Should trace based on numbers. Unknown Error'))
            numbers += 1
            continue
        case_numbers.append(numbers)
        case_names.append(case_name)



        # Description Tab
        description_tab = soup.find_all(class_ = 'details-wrapper')[0]
        ## Intro 랑 Description만 필요
        contents_names = description_tab.find_all(class_ = 'field__label')
        contents_des = description_tab.find_all(class_ = 'field__item')
        descripts = []

        i = 0
        while i < len(contents_names):
            if contents_names[i].text.strip() in ['Intro', 'Description']:
                descripts.append((contents_names[i].text.strip(), contents_des[i].text.strip()))
            
            i += 1

        descrip_tab.append(descripts)

        #SDGS & Targets Tab
        sdgs_targets_tab = soup.find_all(class_ = 'details-wrapper')[1]

        GOALS = [sdgs_targets_tab.find_all(class_ = 'taxonomy-term container vocabulary-goals')[n].find(class_ = 'goal-title').text.strip() for n in range(len(sdgs_targets_tab.find_all(class_ = 'taxonomy-term container vocabulary-goals')))]
        sdgs_tab.append(GOALS)



        # Deliverables&Timeline tab
        deliverable_tab = soup.find_all(class_ = 'details-wrapper')[3]
        
        
        # time_line = deliverable_tab.find_all(class_ = 'datetime')
        table_rows = deliverable_tab.find_all(class_ = 'paragraph paragraph--type--description-of-deliverable paragraph--view-mode--default')
        i = 0
        delivers = []
        while i < len(table_rows):
            time_line = table_rows[i].find(class_ = 'field field--name-field-date-of-deliverable field--type-datetime field--label-hidden field__item')
            try:
                time_line_text = time_line.text.strip() 
            except:
                time_line_text = np.NaN
            time_description = table_rows[i].find(class_ = 'clearfix text-formatted field field--name-field-description-of-deliverable field--type-text-long field--label-hidden field__item')
            try:
                time_description_text = time_description.text.strip()
            except:
                time_description_text = np.NaN

            delivers.append((time_line_text, time_description_text))
            i += 1
        deliver_tab.append(delivers)


        # Resources Mobilized
        resources_tab = soup.find_all(class_ = 'details-wrapper')[4]
        resource_table_rows  = resources_tab.find_all(class_ = 'paragraph paragraph--type--resources-devoted-to-delivery paragraph--view-mode--default')

        i = 0
        resources = [] 
        while i < len(resource_table_rows):
            the_resource = resource_table_rows[i].find(class_ = 'field field--name-field-type field--type-list-string field--label-hidden field__item')
            try:
                the_resource_text = the_resource.text.strip()
            except:
                the_resource_text = np.NaN
            
            resource_description = resource_table_rows[i].find(class_ = 'field field--name-field-details field--type-string field--label-hidden field__item')
            try:
                resource_description_text = resource_description.text.strip()
            except:
                resource_description_text = np.NaN

            resources.append((the_resource_text, resource_description_text))

            i += 1
        
        resource_mobilized_tab.append(resources)


        # Progress Reports
        progress_report_tqb = soup.find_all(class_ = 'details-wrapper')[5]
        progress_titles = progress_report_tqb.find_all(class_ = 'views-field views-field-title')
        progress_status = progress_report_tqb.find_all(class_ = 'views-field views-field-field-progress-status')
        progress_filing = progress_report_tqb.find_all(class_ = 'views-field views-field-created')

        i = 1
        progress_info = []
        while i < len(progress_titles):
            progress_info.append((progress_titles[i].text.strip(), progress_status[i].text.strip(), progress_filing[i].text.strip()))
            i += 1

        progress_tab.append(progress_info)


        
        ## Right_hand Side
        right_side = soup.find(class_ = 'partnerships-basic-information-block container mt-5')
        containers = right_side.find_all(class_ = 'views-element-container')
        rights = []

        for x in containers:
            temp = []
            if x.text.strip().split('\n')[0] in ['Region', 'Geographical coverage', 'Countries', 'Entity', 'Timeline']:
                pure = [h for h in x.text.strip().split('\n') if (len(h) > 0) & (h != ' ')]
                temp.append((pure[0], pure[1:]))
            if len(temp) != 0:
                rights.append(temp)

        if len(rights) != 0:
            right_hand_info.append(rights)
        else:
            right_hand_info.append(np.nan)

        driver.back()
        time.sleep(1.5)
        # l += 1
        numbers += 1
    if numbers == total_number:
        break    
    # next_page_path = [x for x in driver.find_elements(By.CLASS_NAME, 'page-item') if x.text.strip() == str(k+1)][0]
    # gpt_click(driver, next_page_path)
    url = f'https://sdgs.un.org/partnerships/browse?page={k}'
    driver.get(url)
    time.sleep(1)
    k += 1



# 우측에서 
## Entity
## Timeline 추가 (Start date, Completion date 가 있는데 completion date 가 없으면 없다고)
## Completion 이 아닌 하나의 다른 종류의 date가 하나 있고, 그거가 관측되면 새로운 칼럼으로.


#
## Goal 의 dummy화
## Country 도 더미화
## 된거랑 아닌거랑.


In [ ]:
error_nums = []
for x,y in errors:
    error_nums.append(x)
error_nums

In [ ]:
len(errors)

In [ ]:
print(len(case_numbers))
print(len(case_names))
print(len(descrip_tab))
print(len(sdgs_tab))
print(len(deliver_tab))
print(len(resource_mobilized_tab))
print(len(progress_tab))

print(len(right_hand_info))

In [ ]:
series_storage = []

i = 0
j = 0
k = 0
for num in collection_num:
    if num in error_nums:
        case_num = errors[k][0]
        case = errors[k][1]
        description_intro = 'error'
        description_des = 'error'
        sdgs = 'error'
        delivery_date = 'error'
        delivery_des = 'error'
        resource_name = 'error'
        resource_des = 'error'
        progress_t = 'error'
        progress_s = 'error'
        progress_f = 'error'
        region = 'error'
        geo_cover = 'error'
        countries = 'error'
        entity = 'error'
        timeline = 'error'
        k += 1
    else:
        case_num = case_numbers[j]
        case = case_names[j]

        if len(descrip_tab[j]) == 0:
            description_intro = np.nan
            description_des = np.nan
        elif len(descrip_tab[j]) == 1:
            if descrip_tab[j][0][0] == 'Intro':
                description_intro = descrip_tab[j][0][1]
                description_des = np.nan
            elif descrip_tab[j][0][0] == 'Description':
                description_intro = np.nan
                description_des = descrip_tab[j][0][1]
        else:
            if descrip_tab[j][0][0] == 'Intro':
                description_intro = descrip_tab[j][0][1]
                description_des = descrip_tab[j][1][1]
            elif descrip_tab[j][0][0] == 'Description':
                description_intro = descrip_tab[j][1][1]
                description_des = descrip_tab[j][0][1]

        sdgs = sdgs_tab[j]

        delivery_date = []
        delivery_des = []

        for x,y in deliver_tab[j]:
            delivery_date.append(x)
            delivery_des.append(y)

        resource_name = []
        resource_des = []

        for x,y in resource_mobilized_tab[j]:
            resource_name.append(x)
            resource_des.append(y)

        progress_t = []
        progress_s = []
        progress_f = []
        
        for x,y,z in progress_tab[j]:
            progress_t.append(x)
            progress_s.append(y)
            progress_f.append(z)
        

        right_dict = {}

        for element in right_hand_info[j]:
            target = element[0]
            right_dict[target[0]] = target[1]
        
        for word in ['Region', 'Geographical coverage', 'Countries']:
            if word not in right_dict.keys():
                right_dict[word] = np.NaN
            else:
                pass

        region = right_dict['Region']
        geo_cover = right_dict['Geographical coverage']
        countries = right_dict['Countries']
        entity = right_dict['Entity']
        timeline = right_dict['Timeline']

        

        j += 1

        

        
    # i += 1
                





    series = pd.Series({'Case_Num':case_num, 'Case_Name' : case, 'Description_Intro' : description_intro, 'Description_Description' : description_des,
               'SDGS&TARGETS' : sdgs, 'Delivery_Date' :  delivery_date, 'Delivery_Description' : delivery_des, 'Resource_Name': resource_name,
               'Resource_Description': resource_des, 'Progress_Title': progress_t, 'Progress_Status': progress_s, 'Progress_Filing': progress_f,
               'Region' : region, 'Geographical Coverage': geo_cover, 'Countries':countries, 'Entity':entity, 'Timeline':timeline})

    series_storage.append(series)


In [ ]:
data = pd.DataFrame(series_storage)
data

In [ ]:
len(data.columns)

In [ ]:
len(data.iloc[1,8])

In [ ]:
i = 0

while i < len(data):
    j = 0
    while j < len(data.columns):
        if (type(data.iloc[i,j]) == list):
            if len(data.iloc[i,j]) == 0:
                data.iloc[i,j] = np.NaN
        j += 1
    i += 1
data

In [ ]:
data.iloc[4,14]

In [ ]:
i = 0

while i < len(data):
    j = 0
    while j < len(data.columns):
        if data.iloc[i,j] == ['\t\t\t\t\t\t\tN/A']:
            data.iloc[i,j] = np.NaN
        j += 1
    i += 1
data

In [ ]:
reference_table[reference_table['Case_Num'].isin(list(data['Case_Num'])) == False]

In [ ]:
reference_table2 = reference_table[reference_table['Case_Num'].isin(list(data['Case_Num'])) == False]
reference_table2

In [ ]:
len(data) + len(reference_table2)

In [ ]:
data2 = pd.concat([reference_table2, data])
data2

In [ ]:
data3 = data2.sort_values(by = 'Case_Num')

In [ ]:
data3.reset_index(inplace=True, drop=True)
data3

In [ ]:
len(data3.columns)

In [ ]:
i = 0

while i < len(data3):
    j = 0
    while j < len(data3.columns):
        if data3.iloc[i,j] == ['\t\t\tN/A']:
            data3.iloc[i,j] = np.NaN
        j += 1
    i += 1
data3

In [ ]:
data3.to_excel('SDGS_on_process.xlsx', index=False)

In [ ]:
import pandas as pd

In [ ]:
the_data =pd.read_excel('SDGS_on_process.xlsx', index_col=False)
the_data

In [ ]:
driver.quit()

In [ ]:
import ast
import json

def convert_to_list(value):
    if isinstance(value, str):
        try:
            # JSON 형식일 경우
            return json.loads(value)
        except json.JSONDecodeError:
            try:
                # 쉼표로 구분된 문자열일 경우
                return ast.literal_eval(value)
            except (ValueError, SyntaxError):
                # 문자열 그대로 반환
                return value
    return value


In [ ]:
list_form = ['SDGS&TARGETS', 'Delivery_Date', 'Delivery_Description',
       'Resource_Name', 'Resource_Description', 'Progress_Title',
       'Progress_Status', 'Progress_Filing', 'Region', 'Geographical Coverage',
       'Countries', 'Entity', 'Timeline']

In [ ]:
for x in list_form:
    the_data[x] = the_data[x].apply(convert_to_list)

In [ ]:
the_data['SDGS&TARGETS'][0]

In [ ]:
import numpy as np

In [ ]:
GOAL1 = []
GOAL2 = []
GOAL3 = []
GOAL4 = []
GOAL5 = []
GOAL6 = []
GOAL7 = []
GOAL8 = []
GOAL9 = []
GOAL10 = []
GOAL11 = []
GOAL12 = []
GOAL13 = []
GOAL14 = []
GOAL15 = []
GOAL16 = []
GOAL17 = []

Goal_dict = {'Goal 1' : GOAL1, 'Goal 2' : GOAL2, 'Goal 3': GOAL3,'Goal 4': GOAL4, 'Goal 5': GOAL5, 'Goal 6': GOAL6, 'Goal 7': GOAL7, 'Goal 8': GOAL8, 'Goal 9': GOAL9, 'Goal 10': GOAL10, 'Goal 11': GOAL11, 'Goal 12': GOAL12, 'Goal 13': GOAL13, 'Goal 14': GOAL14, 'Goal 15': GOAL15, 'Goal 16': GOAL16, 'Goal 17': GOAL17}


for goals in list(the_data['SDGS&TARGETS']):
    if type(goals) == float:
        for key in list(Goal_dict.keys()):
            Goal_dict[key].append(np.NaN)
    elif goals == 'error':
        for key in list(Goal_dict.keys()):
            Goal_dict[key].append('error')
    else:
        for key in list(Goal_dict.keys()):

            if key in goals:
                Goal_dict[key].append(1)
            else:
                Goal_dict[key].append(0)
goal_dummies = pd.DataFrame(Goal_dict)

In [ ]:
goal_dummies

In [ ]:
indexa = 0
odd = []
nons = []
for countries in list(the_data['Countries']):
    if type(countries) == float:
        nons.append(indexa)
        indexa += 1
        continue
    if type(countries) != list:
        odd.append(indexa)
    indexa += 1
len(odd)

In [ ]:
odd

In [ ]:
nons

In [ ]:
list(the_data['Countries'])[4395]

In [ ]:
the_data['Countries'][0][0]

In [ ]:
import numpy as np

countries_dict = {}
number = 0

for countries in list(the_data['Countries']):
    # Handle NaN cases
    if type(countries) == float:
        for key in list(countries_dict.keys()):
            countries_dict[key].append(np.NaN)
        number += 1
        continue
    
    # Handle 'error' cases
    elif countries == 'error':
        for key in list(countries_dict.keys()):
            countries_dict[key].append('error')
        number += 1
        continue
    
    # Handle valid country lists
    else:
        for country in countries:
            if country in countries_dict:
                countries_dict[country].append(1)
            else:
                # Add new country with 0's for previous entries
                countries_dict[country] = [0] * number
                countries_dict[country].append(1)
        
        # Ensure all existing keys have the same length
        for key in countries_dict.keys():
            if len(countries_dict[key]) < number + 1:
                countries_dict[key].append(0)
    
    number += 1


In [ ]:
country_dummies = pd.DataFrame(countries_dict)
country_dummies

In [ ]:
i = 0
while i < len(country_dummies):
    if i in nons:
        j = 0
        while j < len(country_dummies.columns):
            country_dummies.iloc[i,j] == np.NaN
            j += 1
    
    elif i in odd:
        j = 0
        while j < len(country_dummies.columns):
            country_dummies.iloc[i,j] == 'error'
            j += 1        
        
    i += 1

In [ ]:
country_dummies.iloc[nons[0],:]

In [ ]:
the_data2 = pd.concat([the_data, goal_dummies], axis = 1)
the_data2

In [ ]:
the_data3 = pd.concat([the_data2, country_dummies], axis = 1)
the_data3

In [ ]:
the_data3.to_excel('SDGS_with_dummies.xlsx')

In [ ]:
the_data

In [ ]:
the_data.to_excel('SDGS_data.xlsx')